In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [14]:
import torch 
from torch import nn 

# Assuming you have a list 'frames_features' containing feature vectors for 8 frames
frames_features = torch.randn((16, 8, 10)).cuda()
frames_features_list = [list(frames_features[i]) for i in range(16)]
# Number of frames
num_frames = len(frames_features_list)

# Number of classifiers (assuming 8 classifiers for 8 frames)
num_classifiers = num_frames

input_dim = 10
num_classes_per_classifier = 150  #

classifiers = [nn.Linear(input_dim, num_classes_per_classifier).cuda() for _ in range(num_classifiers)]

batch_framewise_logits_list = []
for batch_idx in range(16):
    framewise_logits_list = []
    for frame_idx in range(8):
        frame_feature = frames_features[batch_idx][frame_idx]
        classifier = classifiers[frame_idx]
        # Forward pass through the classifier
        logits = classifier(frame_feature)
        # Store the logits for the current frame
        framewise_logits_list.append(logits)
    batch_framewise_logits_list.append(framewise_logits_list)



In [18]:
final = torch.stack([torch.stack(batch_framewise_logits_list[i]) for i in range(len(batch_framewise_logits_list))])
final_ = final.view(-1, 150)
final_.shape

torch.Size([128, 150])

In [28]:
class_indices = torch.randint(0, 150, (16,))
class_indices = class_indices.repeat_interleave(8).cuda()

In [29]:
loss_fn = nn.CrossEntropyLoss()

# Compute the loss
loss = loss_fn(final_, class_indices)
loss

tensor(5.0967, device='cuda:0', grad_fn=<NllLossBackward0>)

In [23]:
original_tensor = torch.arange(16)

# Duplicate each element 7 more times to create a new tensor of size (128,)
new_tensor = original_tensor.repeat_interleave(8)

# Now the 'new_tensor' has a size of (128,)
new_tensor

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
         2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
         4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,
         6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,
         9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11,
        11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13,
        13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15,
        15, 15])

In [12]:
a = torch.rand((16, 8, 10)).cuda()
b = [list(a[i]) for i in range(a.size(0))]
b[0][2]

tensor([0.9770, 0.8260, 0.6012, 0.7240, 0.6263, 0.0738, 0.7951, 0.5541, 0.0893,
        0.2656], device='cuda:0')

In [4]:
a = torch.rand((16, 8, 1024))
b = a.permute(1, 0, 2)
b.shape

torch.Size([8, 16, 1024])

In [10]:
datapath = 'data/ccvid/gallery.pkl'

with open(datapath, 'rb') as f:
    content = pickle.load(f)

In [71]:
idx = 1
len(content['data'][idx]['img_paths'])

25

In [15]:
pid_container = set()
for item in content['data']:
    pid_container.add(item['p_id'])

pid_container = sorted(pid_container)
pid2label = {pid:label for label, pid in enumerate(pid_container)}

for item in content['data']:
    cur_pid = item['p_id']
    new_pid = pid2label[cur_pid]
    item['p_id'] = new_pid


In [19]:
with open(datapath, 'wb') as f:
    pickle.dump(content, f)

In [22]:
for item in content['data']:
    del item['tracklet_id']

In [23]:
with open(datapath, 'wb') as f:
    pickle.dump(content, f)

In [30]:
import pickle, os
import os.path as osp

old_path = '/media/dustin/DATA/Research/Video-based ReID'
new_path = '/project/shah/dustin/VideoDatasets'

# for dataset in os.listdir('data/vccr'):
#     dataset_path = osp.join('data', dataset)
# for file in os.listdir('data/ccvid'):
#     mode = file.split('.')[0]
file_path = 'data/ccvid/train.pkl'
with open(file_path, 'rb') as f:
    content = pickle.load(f)
paths = []
for item in content['data']:
    for i in item['img_paths']:
        path = i[0]
        paths.append(path)
    item['img_paths'] = paths
# new_file_path = osp.join('data/ccvid', 'train_.pkl')
with open(file_path, 'wb') as f:
    pickle.dump(content, f)
    
print('done')

done
